# Frozen Bot
Heladerías Frozen SRL nos solicita la construcción de un bot para la toma de
pedidos. Es nuestra responsabilidad el desarrollo de funciones auxiliares que le darán al bot la capacidad de desenvolverse en la conversación.

Flujo de diálogo

El siguiente diagrama ejemplifica el diálogo del bot:

![Flujo de diálogo](images/flujo.jpg)

Como se puede observar en el diagrama, en determinados momentos, el bot necesitará realizar validaciones externas para evaluar cómo continuar.

A continuación, se define el alcance de esas funciones y se solicita desarrollar las mismas.

## 1: GeoAPI

Completar el método is_hot_in_pehuajo con el siguiente objetivo:
* Consultar la información de clima y devolver True si la temperatura actual
supera los 28 grados celsius o False caso contrario. Esto implica incluso
devolver False ante cualquier excepción http.

API Información de clima: 
Link a documentacion: https://openweathermap.org/current#geo

Comenzamos importando requests para realizar solicitudes HTTP a la API de información climática. Definimos la clase GeoAPI y el método is_hot_in_pehuajo dentro de ella.

Nota: puedes instalar requests desde el propio notebook con :
%pip install requests ó !pip install requests

También voy a instalar logging para manejar exepciones y que no se vea afectado el retorno de la función.

%pip install logging

In [2]:
import logging
import requests

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class GeoAPI:
    """
    Clase para interactuar con la API de información de clima.
    """

    API_KEY = "d81015613923e3e435231f2740d5610b"
    LAT = "-35.836948753554054" 
    LON = "-61.870523905384076"
    
    MAX_TEMP = 28

    @classmethod
    def is_hot_in_pehuajo(cls):
        """
        Consulta la información de clima y devuelve True si la temperatura actual supera los 28 grados Celsius,
        o False en caso contrario. Devuelve False ante cualquier excepción http.

        :return: True si la temperatura es mayor a 28 grados, False en caso contrario.
        :rtype: bool
        """
        try:
            url = f"https://api.openweathermap.org/data/2.5/weather"
            parameters = {"lat": cls.LAT, "lon": cls.LON, "appid": cls.API_KEY, "units": "metric"}
            response = requests.get(url, parameters)

            if response.status_code == 200:
                data = response.json()
                temperature = data["main"]["temp"]

                # Registrar mensaje informativo
                logger.info(f"La temperatura actual en Pehuajó es {temperature} grados Celsius")

                return temperature > cls.MAX_TEMP
            else:
                # Registrar mensaje de error
                logger.error("No se pudo obtener la información del clima")

                return False
        except requests.exceptions.RequestException:
            # Registrar mensaje de excepción
            logger.exception("Ocurrió un error al realizar la solicitud HTTP")

            return False
        
# Llamar al método is_hot_in_pehuajo para verificar si hace calor en Pehuajó
result = GeoAPI.is_hot_in_pehuajo()

# Imprimir el resultado
if result:
    print("¡Hace calor en Pehuajó!")
else:
    print("No hace calor en Pehuajó.")


2023-07-09 00:03:13,271 - DEBUG - Starting new HTTPS connection (1): api.openweathermap.org:443
2023-07-09 00:03:14,545 - DEBUG - https://api.openweathermap.org:443 "GET /data/2.5/weather?lat=-35.836948753554054&lon=-61.870523905384076&appid=d81015613923e3e435231f2740d5610b&units=metric HTTP/1.1" 200 489
2023-07-09 00:03:14,549 - INFO - La temperatura actual en Pehuajó es 7 grados Celsius


No hace calor en Pehuajó.


Nota: Si estas utilizando Visual Estudio Code y si no lo hiciste antes, al ejecutar la primera celda con python te debería salir la ventana emrgente siguiente, donde debes instalar ipykernel package:

![instalar ipykernel package](images/ipykernel-package.jpg)